In [89]:
import numpy as np
import pandas as pd
import nltk

In [90]:
book_ratings = pd.read_csv('datasets\\Ratings.csv', delimiter=";", encoding='latin-1')
book_ratings

,User-ID,ISBN,Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [91]:
books = pd.read_csv('datasets\\Books.csv', encoding='latin-1', delimiter=';')
books

,ISBN,Title,Author,Year,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company
...,...,...,...,...,...
271374,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm)
271375,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books
271376,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco
271377,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press


In [92]:
users = pd.read_csv('datasets\\Users.csv',encoding='latin-1', delimiter=';', )
users

C:\Users\sanya\AppData\Local\Temp\ipykernel_12568\2585424410.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  users = pd.read_csv('datasets\\Users.csv',encoding='latin-1', delimiter=';', )


,User-ID,Age
0,1,NaN
1,2,18
2,3,NaN
3,4,17
4,5,NaN
...,...,...
278854,278854,NaN
278855,278855,50
278856,278856,NaN
278857,278857,NaN


In [93]:
book_ratings.rename(columns={"User-ID":"user_id","Rating":"rating"}, inplace=True)

In [94]:
books.rename(columns={"Title":"title", "Author":"author", "Year":"year", "Publisher":"publisher"}, inplace=True)

In [95]:
users.rename(columns={"User-ID":"user_id", "Age":"age"}, inplace=True)

In [96]:
unique_user_ids = book_ratings['user_id'].unique()

In [97]:
book_ratings = book_ratings[book_ratings['user_id'].isin(unique_user_ids)]
book_ratings

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [98]:
rating_with_books = book_ratings.merge(books, on='ISBN')
rating_with_books

,user_id,ISBN,rating,title,author,year,publisher
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
...,...,...,...,...,...,...,...
1031182,276688,0517145553,0,Mostly Harmless,Douglas Adams,1995,Random House Value Pub
1031183,276688,1575660792,7,Gray Matter,Shirley Kennett,1996,Kensington Publishing Corporation
1031184,276690,0590907301,0,Triplet Trouble and the Class Trip (Triplet Tr...,Debbie Dadey,1997,Apple
1031185,276704,0679752714,0,A Desert of Pure Feeling (Vintage Contemporaries),Judith Freeman,1997,Vintage Books USA


In [99]:
number_rating = rating_with_books.groupby('title')['rating'].count().reset_index()
number_rating

,title,rating
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241085,Ã?Ã?lpiraten.,2
241086,Ã?Ã?rger mit Produkt X. Roman.,4
241087,Ã?Ã?sterlich leben.,1
241088,Ã?Ã?stlich der Berge.,3


In [100]:
number_rating.rename(columns= {'rating':'number_of_ratings'}, inplace=True)
number_rating

,title,number_of_ratings
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241085,Ã?Ã?lpiraten.,2
241086,Ã?Ã?rger mit Produkt X. Roman.,4
241087,Ã?Ã?sterlich leben.,1
241088,Ã?Ã?stlich der Berge.,3


In [101]:
final_rating = rating_with_books.merge(number_rating, on='title')
final_rating

,user_id,ISBN,rating,title,author,year,publisher,number_of_ratings
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,60
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,60
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,60
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,60
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,60
...,...,...,...,...,...,...,...,...
1031182,276688,0425150526,0,Death Crosses the Border,Janice Steinberg,1995,Berkley Publishing Group,1
1031183,276688,0449907422,0,Jazz Funeral: A Skip Langdon Novel,Julie Smith,1993,Ballantine Books,1
1031184,276690,0590907301,0,Triplet Trouble and the Class Trip (Triplet Tr...,Debbie Dadey,1997,Apple,1
1031185,276704,0679752714,0,A Desert of Pure Feeling (Vintage Contemporaries),Judith Freeman,1997,Vintage Books USA,1


In [102]:
final_rating.duplicated(['user_id','title'])

0          False
1          False
2          False
3          False
4          False
           ...  
1031182    False
1031183    False
1031184    False
1031185    False
1031186    False
Length: 1031187, dtype: bool

In [103]:
final_rating.drop_duplicates(['user_id','title'], inplace=True)

In [104]:
final_rating.set_index(['user_id', 'title'], inplace=True)
final_rating

ISBN  \
user_id title                                                            
276725  Flesh Tones: A Novel                                034545104X   
2313    Flesh Tones: A Novel                                034545104X   
6543    Flesh Tones: A Novel                                034545104X   
8680    Flesh Tones: A Novel                                034545104X   
10314   Flesh Tones: A Novel                                034545104X   
...                                                                ...   
276688  Death Crosses the Border                            0425150526   
        Jazz Funeral: A Skip Langdon Novel                  0449907422   
276690  Triplet Trouble and the Class Trip (Triplet Tro...  0590907301   
276704  A Desert of Pure Feeling (Vintage Contemporaries)   0679752714   
        Perplexing Lateral Thinking Puzzles: Scholastic...  0806917695   

                                                            rating  \
user_id title                                                        
276725  Flesh Tones: A Novel                                     0   
2313    Flesh Tones: A Novel                                     5   
6543    Flesh Tones: A Novel                                     0   
8680    Flesh Tones: A Novel                                     5   
10314   Flesh Tones: A Novel                                     9   
...                                                            ...   
276688  Death Crosses the Border                                 0   
        Jazz Funeral: A Skip Langdon Novel                       0   
276690  Triplet Trouble and the Class Trip (Triplet Tro...       0   
276704  A Desert of Pure Feeling (Vintage Contemporaries)        0   
        Perplexing Lateral Thinking Puzzles: Scholastic...       5   

                                                                      author  \
user_id title                                                                  
276725  Flesh Tones: A Novel                                      M. J. Rose   
2313    Flesh Tones: A Novel                                      M. J. Rose   
6543    Flesh Tones: A Novel                                      M. J. Rose   
8680    Flesh Tones: A Novel                                      M. J. Rose   
10314   Flesh Tones: A Novel                                      M. J. Rose   
...                                                                      ...   
276688  Death Crosses the Border                            Janice Steinberg   
        Jazz Funeral: A Skip Langdon Novel                       Julie Smith   
276690  Triplet Trouble and the Class Trip (Triplet Tro...      Debbie Dadey   
276704  A Desert of Pure Feeling (Vintage Contemporaries)     Judith Freeman   
        Perplexing Lateral Thinking Puzzles: Scholastic...       Paul Sloane   

                                                            year  \
user_id title                                                      
276725  Flesh Tones: A Novel                                2002   
2313    Flesh Tones: A Novel                                2002   
6543    Flesh Tones: A Novel                                2002   
8680    Flesh Tones: A Novel                                2002   
10314   Flesh Tones: A Novel                                2002   
...                                                          ...   
276688  Death Crosses the Border                            1995   
        Jazz Funeral: A Skip Langdon Novel                  1993   
276690  Triplet Trouble and the Class Trip (Triplet Tro...  1997   
276704  A Desert of Pure Feeling (Vintage Contemporaries)   1997   
        Perplexing Lateral Thinking Puzzles: Scholastic...  1997   

                                                                           publisher  \
user_id title                                                                          
276725  Flesh Tones: A Novel                                        Ballantine Books   
23

In [107]:
book_pivot = final_rating.pivot_table(values='rating', columns='user_id', index='title', fill_value=0)
book_pivot

c:\Users\sanya\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\reshape\reshape.py:125: RuntimeWarning: overflow encountered in scalar multiply
  num_cells = num_rows * num_columns


IndexError: index 731270092 is out of bounds for axis 0 with size 731240150

In [ ]:

# book_pivot = final_rating.pivot_table(columns='user_id', index='title', values="rating")
book_pivot.fillna(0, inplace=True)

In [ ]:
book_pivot

In [ ]:
from scipy.sparse import csr_matrix
book_sparse = csr_matrix(book_pivot)

In [ ]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='auto')
model.fit(book_sparse)

In [ ]:
distances, suggestions = model.kneighbors(book_pivot.iloc[69, :].values.reshape(1, -1)) 

In [ ]:
book_pivot.iloc[69].name

In [ ]:
distances

In [ ]:
suggestions

In [ ]:
book_pivot.index[suggestions][0]